## Instalação de Deplyoyment do 5G Core (5GC) do OAI
## Objetivos
As metas desse tutorial são ajudar o usuário a:
- Instalar e configurar o 5GC do OAI em uma maquina virtual Linux Ubuntu 20.04 LTS (Focal Fossa)
- Fazer o deployment do 5GC Core, observar seu status 
- Instalar o Portainer para verificar e interagir com os pods do 5GC

At the end of this tutorial, the goal is to run the simulated OAI scenario, which consists of the 5G core (5GC), gNB in CU/DU split mode, and user equipment (UE), according to the image below: 

![5GC_01](./FIGS/H01_5GCore_UNI_III/5GC_deploy.jpg)

Now, We are going to install, configure, deploy and interact to 5GC.


## General Instructions

In this task you will install and configure the core in its basic architecture according to the image below:

![5GC_01](./FIGS/H01_5GCore_UNI_III/5GC_deploy_pods_01.png)

**Minimum hardware requirements for OAI UE, according to doc/NR_SA_Tutorial_COTS_UE.md · develop · oai / openairinterface5G · GitLab (eurecom.fr):**
- Ubuntu 20.04.5 LTS;
- CPU: 4 cores x86_64;
- 8 GB RAM.

**Minimum hardware requirements for OAI 5GC and OAI gNB, according to doc/NR_SA_Tutorial_COTS_UE.md · develop · oai / openairinterface5G · GitLab (eurecom.fr):**
- Ubuntu 20.04.5 LTS;
- CPU: 8 cores x86_64 3.5 GHz;
- 32 GB RAM.

**However, if you do not have such hardware specifications, you may use the one below for both devices (tested for us, you may face some lags, but it works):**
- Ubuntu 20.04.5 LTS;
- CPU: 4 cores x86_64;
- 4 GB RAM.

## 5GC Installation

In a Linux Terminal, follow the steps below:

**Step 0:** Setup your VM or Baremetal Machine, after install Useful Software:

sudo su

apt update -y

apt upgrade -y

apt install -y python3-pip git curl cmake tree golang vim net-tools build-essential nfs-common openssh-server

**Step 1:** Install dependencies

sudo apt-get install ca-certificates 

sudo apt-get install curl 

sudo apt-get install gnupg 

sudo apt-get install lsb-release 

**Step 2:** Enable packet forwarding

sudo sysctl net.ipv4.conf.all.forwarding=1

sudo iptables -P FORWARD ACCEPT

**Step 3:** Install docker and docker-compose

sudo rm /etc/apt/sources.list.d/docker.list*

sudo mkdir -p /etc/apt/keyrings 

curl -fsSL https://download.docker.com/linux/ubuntu/gpg | sudo apt-key add -

sudo add-apt-repository "deb [arch=amd64] https://download.docker.com/linux/ubuntu $(. /etc/os-release; echo "$UBUNTU_CODENAME") stable"

sudo chmod a+r /etc/apt/keyrings/docker.gpg

sudo apt-get update

sudo apt-get install -y docker-ce docker-ce-cli containerd.io docker-compose-plugin

sudo curl -L https://github.com/docker/compose/releases/download/1.29.2/docker-compose-`uname -s`-`uname -m` -o /usr/local/bin/docker-compose

sudo chmod +x /usr/local/bin/docker-compose

**Step 4:** Pull the images from Docker Hub

sudo docker pull oaisoftwarealliance/oai-amf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-nrf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-smf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-udr:v1.5.0

sudo docker pull oaisoftwarealliance/oai-udm:v1.5.0

sudo docker pull oaisoftwarealliance/oai-ausf:v1.5.0

sudo docker pull oaisoftwarealliance/oai-spgwu-tiny:v1.5.0

sudo docker pull oaisoftwarealliance/trf-gen-cn5g:latest

**Step 5:** Tag Docker Images

sudo docker image tag oaisoftwarealliance/trf-gen-cn5g:latest trf-gen-cn5g:latest

sudo docker image tag oaisoftwarealliance/oai-amf:v1.5.0 oai-amf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-nrf:v1.5.0 oai-nrf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-smf:v1.5.0 oai-smf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-udr:v1.5.0 oai-udr:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-udm:v1.5.0 oai-udm:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-ausf:v1.5.0 oai-ausf:v1.5.0

sudo docker image tag oaisoftwarealliance/oai-spgwu-tiny:v1.5.0 oai-spgwu-tiny:v1.5.0

**Step 6:** Clone OpenAirInterface 5G Core repository

git clone https://gitlab.eurecom.fr/oai/cn5g/oai-cn5g-fed.git

cd oai-cn5g-fed

git checkout -f v1.5.0

cd ..


## 5GC Configuration

In a Linux Terminal, follow the steps below:

**Step 1:** Configuration

The configuration will be performed using docker-compose. First, run the command below to download the OAI core docker-compose file. We consider that <OAI_PATH> (path that you install OAI) is **/home/dco1020** (The file will already be placed in the correct path (<OAI_PATH>/oai-cn5g-fed/docker-compose/).

git clone https://github.com/vicentesousa/DCO1020.git

cp ./DCO1020/CODES/H01_5GCore_UNI_III/docker-compose-basic-nrf.yaml  /home/dco1020/oai-cn5g-fed/docker-compose/docker-compose-basic-nrf.yaml

Then download the database used for this deployment. This database contains important information such as data for gNB and UE authentication. The command below will download and place the file already in the correct path (PATH_TO/oai-cn5g-fed/docker-compose/database/).

wget -O PATH_TO/oai-cn5g-fed/docker-compose/database/oai_db.sql https://projetos.imd.ufrn.br/reginalab/sdran-openairinterface/-/raw/main/openairinterface5g/openairinterface-conf/oai_db.sql?inline=false

**Step 2:** Synchronizing the tutorials

cd PATH_TO/oai-cn5g-fed/

sudo ./scripts/syncComponents.sh

sudo git submodule deinit --all --force

sudo git submodule init

sudo git submodule update

**Step 3:** Create the bridge 5GC

The command below will then create a subnet that will provide communication between the containers in the core.

sudo docker network create --driver=bridge --subnet=192.168.70.128/26 -o "com.docker.network.bridge.name"="demo-oai" demo-oai-public-net


## Deployment


## Verificação do cluster Kubernetes e seus Pods com o Portainer

**Passo 1**: Baixe e instale o VirtualBox em seu computador (para acessar a página de Download [acesse aqui](https://www.virtualbox.org/). Em 2023, você pode encontrar a versão VirtualBox 7.0.12.

![Virtual1](./FIGS/H00_virtualbox/virtual_box_novo.png)

**Passo 2**: Baixe a iso do Ubuntu 20.04.6 LTS (Focal Fossa). Algumas telas a seguir são exibidas para a versão 20.04.3,  mas a essência da instalação é a mesma. Para acessar essa versão do Ubuntu navegue até **see our alternative downloads**. O arquivo tem o seguinte nome: **ubuntu-20.04.6-desktop-amd64.iso**. Esse download pode demorar, pois a iso tem 4.1 GB!

- Versão 64 Bits: [https://ubuntu.com/download/desktop](https://ubuntu.com/download/desktop)

![fig_ubuntu1](./FIGS/H00_virtualbox/ubuntu_novo.png)

**Passo 3**: Após instalar o Virtualbox e baixar a iso, abra o VirtualBox e clique em **Novo**, e depois na opção **Criar** de acordo com a imagem. Dê um nome a sua máquina, escolha o local que ela será gravada, a imagem, o tipo e a versão da iso. **Não esqueça de escolher para Pular a Instalação Dessasistida!**

![fig_Virtual_1](./FIGS/H00_virtualbox/virtual_box1_novo.png)

**Passo 4**: Após clicar em **Próximo**, escolha a memória que deseja alocar, o ideal é que seja a metade da memória de sua máquina hospedeira (recomendo pelo menos **8 GB**). Escolha também o número de processadores. O ideal é que sejam 4 CPUs, mas isso dependerá da capacidade de sua máquina. 

![fig_Virtual_3](./FIGS/H00_virtualbox/virtual-box-memoria_novo.png)

**Passo 5**: Após clicar em **Próximo**, especifique **100 GB** de espaço em HD (se tiver pouco espaço em HD, pode criar uma máquina de **60 ou 40 GB**).

![fig_VM3_novo](./FIGS/H00_virtualbox/virtual_box3_novo.png)

**Passo 6**: Na próxima tela, reveja suas configurações e clique em **Finalizar**. Sua máquina foi criada e está pronto para ser inicializada.


**Passo 20 (muito importante)**: Instale o VirtualBox Guest Additions. Ele possibilitará que sua máquina seja aberta em tela cheia, que você consiga copiar e colar texto e arquivos da máquina hospedeira para máquina virtual. 

 - Abra um terminal na sua máquina virtua Linux digitando Crtl+Alt+T
 - No terminal, digite:

sudo apt install -y build-essential dkms linux-headers-$(uname -r)

Depois sigas as instruções de [instalação](https://www.tecmint.com/install-virtualbox-guest-additions-in-ubuntu/#:~:text=VirtualBox%20Guest%20Additions%20are%20a,and%20usability%20of%20guest%20systems.&text=Easy%20mouse%20pointer%20integration.).

Depois de reiniciar sua máquina virtual, você verá uma menu com várias opções, entre elas a de maximizar o VirtualBox para ocupar toda a tela. Ao maximizar, o mesmo menu aparecerá no canto inferior.

![fig_VM3](./FIGS/H00_virtualbox/menu_vbox.png)
